In [7]:
!pip install gensim

In [8]:
#libraries 
import numpy as np
import pandas as pd
import re

#preprocessing
import string

#translation 
from nltk.translate.bleu_score import sentence_bleu
from gensim import models

#ignore warnings
import warnings
warnings.filterwarnings("ignore")

#visualization 
import seaborn as sns

C:\Users\Diana\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


Base: https://medium.com/@adriensieg/text-similarities-da019229c894

In [9]:
df = pd.read_csv(r"C:\Users\Diana\Dropbox\My PC (DESKTOP-LKGC4KF)\Documents\GitHub\textmining_project\scores.csv")
df.head(4)

,source,reference,translation,z-score,avg-score,annotators
0,"Ihr Zeitlupentempo maßen sie, als sie vor Spit...",Her timeless pace measures them when they equi...,Their slow speed was measured by researchers o...,-0.345024,76.0,1
1,"Er sagte, dass die Bereiche ruhige Treffpunkte...",He said the areas offer quiet meeting points b...,He said the spaces provided calm meeting point...,0.903800,97.5,2
2,Für die Geschäftsleute an der B 27 ist es nur ...,"For businessmen at the B 27, it's only a small...",This is only a small consolation for businesse...,0.700503,94.0,1
3,Diese Fähigkeit sei möglicherweise angeboren o...,This ability may be born or developed with gen...,"This ability may be innate, or may develop as ...",-1.256572,51.5,2


In [10]:
df.shape

(21704, 6)

In [11]:
df.describe()

,z-score,avg-score,annotators
count,21704.000000,21704.000000,21704.000000
mean,0.000898,71.852890,1.502995
std,0.855140,26.348469,0.810923
min,-5.806322,0.000000,1.000000
25%,-0.499574,56.000000,1.000000
50%,0.216756,79.000000,1.000000
75%,0.640273,94.000000,2.000000
max,2.535434,100.000000,9.000000


In [12]:
pd.pivot_table(df, values = 'annotators', index =['translation'], aggfunc = 'count')

,annotators
translation,
"""A Generation Is Protesting"" in Ethiopia, Long a U.S. Ally",7
"""A Square Meal"" is a feast of historical tidbits.",4
"""A cultural change, a mental change, a physical change,"" Bam said.",4
"""A lot of former customers got priced out of Williamsburg too,"" he said.",5
"""A lot of the stuff I get is late 60s, early 70s, things that came out when I was 11 or 12, things I was probably a bit too young to get,"" he says.",10
...,...
"You're not singing by yourself - and in a group of 50 people singing, who'll hear if a note's out here or there?",9
YouTube Gaming is owned by Google's parent company Alphabet.,9
Young people are particularly vulnerable to this.,5


In [13]:
reference =df['reference']
translation = df['translation']

## Preprocessing 

In [14]:
df.isnull().sum()

source         0
reference      0
translation    0
z-score        0
avg-score      0
annotators     0
dtype: int64

In [15]:
def preprocessing(column):
    pp_column = []
    for sentence in column:
        #exclude punctuation 
        sentence = re.sub(r'[^A-Za-z0-9]',' ',sentence)
        #transform everything in lowercase
        sentence = sentence.lower()
        pp_column.append(sentence)
        
    processed = pd.Series(pp_column)
    return processed

In [16]:
p_reference = preprocessing(reference)
p_translation = preprocessing(translation)

In [17]:
df_cp = df.copy()

In [18]:
df_cp = pd.concat([df_cp,p_reference.rename('p_reference')], axis=1)

In [19]:
df_cp = pd.concat([df_cp,p_translation.rename('p_translation')], axis=1)

In [20]:
df_cp.head(4)

,source,reference,translation,z-score,avg-score,annotators,p_reference,p_translation
0,"Ihr Zeitlupentempo maßen sie, als sie vor Spit...",Her timeless pace measures them when they equi...,Their slow speed was measured by researchers o...,-0.345024,76.0,1,her timeless pace measures them when they equi...,their slow speed was measured by researchers o...
1,"Er sagte, dass die Bereiche ruhige Treffpunkte...",He said the areas offer quiet meeting points b...,He said the spaces provided calm meeting point...,0.903800,97.5,2,he said the areas offer quiet meeting points b...,he said the spaces provided calm meeting point...
2,Für die Geschäftsleute an der B 27 ist es nur ...,"For businessmen at the B 27, it's only a small...",This is only a small consolation for businesse...,0.700503,94.0,1,for businessmen at the b 27 it s only a small...,this is only a small consolation for businesse...
3,Diese Fähigkeit sei möglicherweise angeboren o...,This ability may be born or developed with gen...,"This ability may be innate, or may develop as ...",-1.256572,51.5,2,this ability may be born or developed with gen...,this ability may be innate or may develop as ...


In [21]:
df_cp.iloc[21,:]

source            Olympia: Dreister Betrug bei Doping-Test
reference           Olympia: threesty fraud in doping test
translation      Olympics: Brazen cheating in doping tests
z-score                                          -0.980437
avg-score                                               39
annotators                                               1
p_reference         olympia  threesty fraud in doping test
p_translation    olympics  brazen cheating in doping tests
Name: 21, dtype: object

In [22]:
teste = df_cp[df_cp['p_translation'] == 'olympics  brazen cheating in doping tests']
teste

,source,reference,translation,z-score,avg-score,annotators,p_reference,p_translation
21,Olympia: Dreister Betrug bei Doping-Test,Olympia: threesty fraud in doping test,Olympics: Brazen cheating in doping tests,-0.980437,39.00,1,olympia threesty fraud in doping test,olympics brazen cheating in doping tests
2240,Olympia: Dreister Betrug bei Doping-Test,Olympia: Brazen fraud with doping test,Olympics: Brazen cheating in doping tests,0.582416,95.25,4,olympia brazen fraud with doping test,olympics brazen cheating in doping tests
4775,Olympia: Dreister Betrug bei Doping-Test,Olympia: Three scams on doping test,Olympics: Brazen cheating in doping tests,-0.085961,73.00,1,olympia three scams on doping test,olympics brazen cheating in doping tests
6867,Olympia: Dreister Betrug bei Doping-Test,Olympia: Trieste cheating on doping test,Olympics: Brazen cheating in doping tests,-0.594918,43.00,1,olympia trieste cheating on doping test,olympics brazen cheating in doping tests
8509,Olympia: Dreister Betrug bei Doping-Test,Olympia: Brazen fraud in doping test,Olympics: Brazen cheating in doping tests,0.758588,95.00,1,olympia brazen fraud in doping test,olympics brazen cheating in doping tests
13797,Olympia: Dreister Betrug bei Doping-Test,Olympia: Dreist fraud in doping test,Olympics: Brazen cheating in doping tests,0.317103,76.00,1,olympia dreist fraud in doping test,olympics brazen cheating in doping tests
15245,Olympia: Dreister Betrug bei Doping-Test,Olympics: triple fraud in doping test,Olympics: Brazen cheating in doping tests,0.099333,77.00,1,olympics triple fraud in doping test,olympics brazen cheating in doping tests
21368,Olympia: Dreister Betrug bei Doping-Test,Olympics: Triple fraud in doping test,Olympics: Brazen cheating in doping tests,-1.098754,25.00,1,olympics triple fraud in doping test,olympics brazen cheating in doping tests


## Cosine Similarity 

REF: https://www.machinelearningplus.com/nlp/cosine-similarity/

The cosine similarity is advantageous because even if the two similar documents are far apart by the Euclidean distance (due to the size of the document), chances are they may still be oriented closer together. The smaller the angle, higher the cosine similarity.

In [23]:
#Libraries 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [24]:
count_vectorizer = CountVectorizer()

In [25]:
ref[0]

NameError: name 'ref' is not defined

In [ ]:
cand

In [ ]:
sparse_matrix = count_vectorizer.fit_transform([ref[0],cand])
sparse_matrix

In [ ]:
pd.DataFrame(cosine_similarity(sparse_matrix))

In [ ]:
sparse_matrix6 = count_vectorizer.fit_transform([ref[6],cand])
sparse_matrix6

In [ ]:
pd.DataFrame(cosine_similarity(sparse_matrix6))

**Interpretation** : A cosine value of 0 means that the two vectors are at 90 degrees to each other (orthogonal) and have no match. The closer the cosine value to 1, the smaller the angle and the greater the match between vectors. 

## BERT 

REF:https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/#what-is-bert

https://medium.com/analytics-vidhya/bert-word-embeddings-deep-dive-32f6214f02bf

In [1]:
!pip install transformers

In [3]:
!pip install torch

In [31]:
!pip install pytorch_transformers

In [62]:
!pip install keras

In [64]:
!pip install tensorflow

ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\Users\\Diana\\anaconda3\\Lib\\site-packages\\~umpy\\core\\_multiarray_tests.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.




  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4835 sha256=9226b6a928da9a6feeda4657efb9945a6ef9b40327d20d738ba185705050d345
  Stored in directory: c:\users\diana\appdata\local\pip\cache\wheels\a0\16\9c\5473df82468f958445479c59e784896fa24f4a5fc024b0f501
  Created wheel for wrapt: filename=wrapt-1.12.1-py3-none-any.whl size=19558 sha256=d74f883fac38643a533cbcbdecec774774500e628e96ac1de6ca83827fc2b3af
  Stored in directory: c:\users\diana\appdata\local\pip\cache\wheels\5f\fd\9e\b6cf5890494cb8ef0b5eaff72e5d55a70fb56316007d6dfe73
Successfully built termcolor wrapt
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5


In [65]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

ModuleNotFoundError: No module named 'tensorflow'

In [33]:
import torch
from pytorch_transformers import BertTokenizer
from pytorch_transformers import BertModel
## Load pretrained model/tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased',output_hidden_states=True)

100%|██████████| 440473133/440473133 [01:50<00:00, 4000524.32B/s]


In [50]:
#teste = df_cp[df_cp['p_translation'] == 'olympics  brazen cheating in doping tests']

# Define an input text
text = df_cp[]
# Add the special tokens.
marked_text = "[CLS] " + text + " [SEP]"
# Split the sentence into tokens.
tokenized_text = tokenizer.tokenize(marked_text)
# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Display the words with their indeces.
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

SyntaxError: invalid syntax (<ipython-input-50-8e8d1e9b2072>, line 4)

In [54]:
!pip install nlu

In [56]:
!pip install pyspark

  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767612 sha256=7fe59baa651b54e592c1013eb9055fe04bdaa0db15f56467e9e3c147312b0f09
  Stored in directory: c:\users\diana\appdata\local\pip\cache\wheels\b3\0e\81\264aeed961e43b9f6ba9ec81c8c540d2d7dccc52c6b51cbf22
Successfully built pyspark


In [59]:
import nlu
#nlu.load('Bert').predict('olympics brazen cheating in doping tests')
pipe = nlu.load('pos sentiment emotion bert')
df_cp['text'] = df_cp['p_translation']
# NLU to gives us one row per embedded word by specifying the output level
predictions = pipe.predict(df[['text','label']], output_level='token')
predictions

NameError: name 'pipe' is not defined

In [40]:
import torch
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
# Put the model in "evaluation" mode,meaning feed-forward operation.
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [43]:
#Run the text through BERT, get the output and collect all of the hidden states produced from all 12 layers.
with torch.no_grad():
    outputs = model(tokens_tensor)
# can use last hidden state as word embeddings
    last_hidden_state = outputs[0]
    word_embed_1 = last_hidden_state
# Evaluating the model will return a different number of objects               based on how it's  configured in the `from_pretrained` call earlier. In this case, becase we set `output_hidden_states = True`, the third item will be the hidden states from all layers. See the documentation for more details:https://huggingface.co/transformers/model_doc/bert.html#bertmodel
hidden_states = outputs[2]
# initial embeddings can be taken from 0th layer of hidden states
word_embed_2 = hidden_states[0]
# sum of all hidden states
word_embed_3 = torch.stack(hidden_states).sum(0)
# sum of second to last layer
word_embed_4 = torch.stack(hidden_states[2:]).sum(0)
# sum of last four layer
word_embed_5 = torch.stack(hidden_states[-4:]).sum(0)
# concatenate last four layers
word_embed_6 = torch.cat([hidden_states[i] for i in [-1,-2,-3,-4]], dim=-1)

## Word Mover Distance

REF: https://towardsdatascience.com/word-movers-distance-for-text-similarity-7492aeca71b0


In [116]:
cand_f = 'olympics  brazen cheating in doping tests'.split()

In [104]:
#libraries 
from time import time 
#remove stop words 
from nltk.corpus import stopwords
import os
from gensim import models
import gensim.downloader as api
from pyemd import emd

In [28]:
start_nb = time()

In [79]:
def delete_sw(lista):
    stop_words = stopwords.words('english')
    finalresult = []
    for sentence in lista:
        processed = sentence.split()
        clean = []
        for word in processed:
            if word not in stop_words:
                clean.append(word)
        finalresult.append(clean)
    return finalresult 

In [81]:
clean_ref = delete_sw(ref)

In [117]:
clean_cand = [w for w in cand_f if w not in stop_words]

In [102]:
model = api.load('word2vec-google-news-300')


[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [119]:
distance = model.wmdistance(clean_ref[0],clean_cand)

https://towardsdatascience.com/word-distance-between-word-embeddings-cc3e9cf1d632
https://radimrehurek.com/gensim/auto_examples/tutorials/run_wmd.html
    
The sentence can have no similar words, but by the relevand words in each sentence, it's possible to identify the "semantical distance" between them. 
With this method we can evaluate how the reference of translation is close to the real translation.

In [110]:
clean_ref

[['olympia', 'threesty', 'fraud', 'doping', 'test'],
 ['olympia', 'brazen', 'fraud', 'doping', 'test'],
 ['olympia', 'three', 'scams', 'doping', 'test'],
 ['olympia', 'trieste', 'cheating', 'doping', 'test'],
 ['olympia', 'brazen', 'fraud', 'doping', 'test'],
 ['olympia', 'dreist', 'fraud', 'doping', 'test'],
 ['olympics', 'triple', 'fraud', 'doping', 'test'],
 ['olympics', 'triple', 'fraud', 'doping', 'test']]

In [118]:
clean_cand

['olympics', 'brazen', 'cheating', 'doping', 'tests']

In [121]:
distance

0.8289396995482206

In [123]:
distance6 = model.wmdistance(clean_ref[6],clean_cand)
distance6

0.5927309122713804

**Interpretation**: more similar words between sentences, less distant they are. 